In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' # to print multiple outputs from the same cell
import math
import utils
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from operator import index
from collections import defaultdict
from scipy.stats import pearsonr
from datetime import datetime
#from lingua import Language, LanguageDetectorBuilder

Creates the combined dataset of the cleaned and generalized user and tweet datasets. The file that is created is the merged_dataset_cleaned

In [3]:
users_df = pd.read_csv("dataset/users_dataset_cleaned.csv")
tweets_df = pd.read_csv("dataset/tweets_dataset_cleaned.csv", lineterminator='\n')

In [4]:
if tweets_df.columns[-1] == str("text\r"):
    tweets_df = pd.read_csv("dataset/tweets_dataset_cleaned.csv")

tweets_df.head()

,tweet_id,user_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,created_at,text
0,509354017856950272,327746321,0,0,0,0,0,0,2019-09-11 14:53:55,"If man is a little lower than angels, then ang..."
1,583002592529121280,333722906,1,0,0,0,0,1,2020-04-01 20:27:04,"""@BestWSHHVids: how do you say these words wit..."
2,461498835362013184,2379755827,0,0,0,0,0,1,2019-05-02 13:34:31,@LOLatComedy awsome
3,528808127366692864,466226882,0,0,0,0,0,0,2019-11-04 07:17:37,Stephen Hawkins: i buchi neri non esistono se ...
4,575336690904006656,1355537995,114,0,0,1,0,1,2020-03-11 16:45:31,RT @tibbs_montris: So ready for Wednesday!


In [5]:
merged_df = pd.merge(users_df, tweets_df, on="user_id")

In [6]:
merged_df.head()

,user_id,name,lang,bot,created_at_x,statuses_count,tweet_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,created_at_y,text
0,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76,486718663223828480,0,0,0,0,0,1,2019-07-11 03:49:06,"@4fri2endly0 ""A business that makes nothing bu..."
1,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76,479535357126393856,0,0,0,0,0,0,2019-06-21 08:05:13,"""Happiness is not a station you arrive at, but..."
2,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76,451700895491706880,0,0,0,0,0,0,2019-04-05 12:41:00,Music flow.
3,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76,491920040405397504,0,0,0,0,0,1,2019-07-25 12:17:31,@_SimplyKC follow meeee...
4,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76,460522543628046336,0,0,0,0,0,0,2019-04-29 20:55:05,"""You are the only person on earth who can use ..."


removing tweets made before author account creation date

In [7]:
# Removes any duplicate columns as a result of the merge
#merged_df = merged_df.loc[:,~merged_df.columns.duplicated()]#.copy()  <-- Add this if we have to avoid memory issues down the line
merged_df.rename(columns={"created_at_x" : "account_created"}, inplace=True)
merged_df.rename(columns={"created_at_y" : "tweet_created"}, inplace=True)
rows_to_delete = merged_df[merged_df["account_created"] > merged_df["tweet_created"]].index
merged_df.drop(rows_to_delete, inplace=True)

In [8]:
merged_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10460636 entries, 0 to 10460635
Data columns (total 15 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   user_id          int64 
 1   name             object
 2   lang             object
 3   bot              int64 
 4   account_created  object
 5   statuses_count   int64 
 6   tweet_id         int64 
 7   retweet_count    int64 
 8   reply_count      int64 
 9   favorite_count   int64 
 10  num_hashtags     int64 
 11  num_urls         int64 
 12  num_mentions     int64 
 13  tweet_created    object
 14  text             object
dtypes: int64(10), object(5)
memory usage: 1.2+ GB


In [9]:
merged_df.to_csv("./dataset/merged_dataset_cleaned.csv",index=False) # Removes the counting of the Index rows